### seismicsph example usage

#### Run smoothed particle hydrodynamical simulation

In [ ]:
import seismicsph_collisions as ss
import seismicsph_propagation as sp
import numpy as np
import pandas as pd

Set temporal and spatial resolution for hydrodynamic simulations.

In [ ]:
# set spatial resolution of simulation
dx = 0.01 # spatial resolution in metres

# set simulation run-time and temporal resolution
t = 2. # duration in seconds
dt_out = 0.001 # temporal resolution in seconds
dt = 1e-5 # initial time step used in numerical calculations (must be less than dt_out)

Set up geometry of tank and initial conditions for fluid assuming the 45 degree bend channel geometry described in Turner et al. (2023). 

In [ ]:
# set dimensions of simulation tank (i.e. computation domain)
x = 10. # length in metres
y = 1. # width in metres
z = 1.5 # height in metres

# define function to set fixed locations of ice (boundary) particles
def set_obstacle(x, y, z, tol):
    # set size of smoothing length (relative to actual kernel; tol) in each direction
    xtol, ytol, ztol = tol, tol, tol
    
    # set region for obstacle particles 
    # (xtol, ytol and ztol are added to include locations within a smoothing length of the boundary)
    return np.where((x <= 0.0 + xtol) | (0.5 - ytol <= y) | (y <= -0.5 + ytol) \
            | ((0.0 - xtol <= x) & (x <= 5.1 - np.sqrt(2)*0.2 + y + xtol) & (-0.3 - ytol <= y) & (y <= 0.5 + ytol)) \
            | ((5.1 + y - xtol <= x) & (x <= 10.0 + xtol) & (-0.5 - ytol <= y) & (y <= 0.3 + ytol)) | (z <= 0.0 + ztol))[0]

# define function to set initial locations of fluid particles
def set_fluid(x, y, z, tol):
    # set size of smoothing length (relative to actual kernel; tol) in each direction
    xtol, ytol, ztol = tol, tol, tol
    
    # set region for fluid particles 
    # (xtol, ytol and ztol are subtracted to exclude locations within a smoothing length of the boundary) 
    # tolerances are not critical for the fluid but included for completeness
    return np.where((0.0 + xtol < x) & (x <= 2.0 + xtol) & (-0.5 + ytol < y) & (y < -0.3 - ytol) & (0.0 + ztol < z) & (z <= 1.0 + ztol))[0]

Run smoothed particle hydrodynamical simulation. This may take a long time to run depending on the chosen resolution.

In [ ]:
ss.run_seismic_sph(x, y, z, t, dx, dt, set_fluid, set_obstacle, output_times=np.arange(0, t + dt_out/2, dt_out), filename='bend_45')

#### Create collision catalogue using impulse capturing method

Find properties of collisions with water--ice interface.

In [ ]:
collision_df = ss.seismic_sph_collisions(x, y, z, dx, set_obstacle, filename='bend_45')

In [ ]:
collision_df.to_csv('bend_45_collisions.csv')

#### Propogate waves generated by particle collisions

Set seismometer location relative to tank assuming the standard location described in Turner et al. (2023).

In [ ]:
# define location of seismometer
x_seis = 15. # x location in metres (relative to zero in the simulation; 10 metres from tank centre)
y_seis = 5. # y location in metres
z_seis = 2. # y location in metres)

# define sampling rate and duration to consider waveform
sample_rate = 1./200 # sample rate in seconds; i.e. 1/f
endtime = 4. # waveform end-time in seconds; may be longer than simulation run-time due to wave travel time

# define ratio of S/P wave acceleration amplitudes
sp_amplitude_ratio=0.5
amplitude_type = 'acceleration'

Read-in collision data file; not needed if already read above.

In [ ]:
collision_df = pd.read_csv('bend_45_collisions.csv', sep=',')

Generate waveform data from input collisions. These outputs do not include a correction for the relative densities of water and ice to yield the acceleration measured by a seismometer imbedded in the ice.

In [ ]:
waveform_df = sp.icewater_collisions(x_seis, y_seis, z_seis, collision_df, endtime, sample_rate=sample_rate, \
                sp_amplitude_ratio=sp_amplitude_ratio, amplitude_type=amplitude_type)

Create basic plots of waveform data for total, P and S wave acceleration amplitudes. The three panels in each plot are the x-, y- and z-components of the signal measured at the location of the seismometer.

In [ ]:
sp.icewater_waveforms(waveform_df, amplitude_type=amplitude_type)